### SQLAlchemy - Core API

Based on the SQLAlchemy <a href="https://docs.sqlalchemy.org/en/14/tutorial/">documentation</a>.

Please, check the [ex1_sqlalchemy.ipynb](ex1_sqlalchemy.ipynb) for an general introduction to the use of SQLAlchemy.

---

**NOTE:** this code is intended to show hot to use the **Core** API for creating and manipulating databases in SQLAlchemy. In [ex3_sqlalchemy_ORM.ipynb](ex3_sqlalchemy_ORM.ipynb), we discuss how to use the **ORM** API for performing the same tasks. Most of the time, you can choose only one approach to work with databases in SQLAlchemy.

---
**OPTIONAL**

Two sections are labeled **optional** in this notebook. Uncomment them (using `Ctrl`  or `Command + /`) if you want to experiment with hybrid approach and table reflection.

There is an **hybrid approach** combining Core and ORM objects. If you want to experiment with that, run the following sections (in this order):
- Core: Setting up MetaData with Table objects
- Core: Declaring primary and foreign key constraints
- [OPTIONAL] Combining Core Table declarations with ORM declarative

**Table Reflection** is another way of declaring tables in SQLAlchemy. For experimenting with that, you **should** run the following sections (in this order):
- Core - Setting up MetaData with Table objects
- Core - Declaring primary and foreign key constraints
- [OPTIONAL] Table Reflection 

Also, remember to **Restart the runtime** every time you change from one approach to another, so you can clean all `Table` objects from the memory-hosted database.

In [1]:
# Uncomment for installing the SQL Alchemy library.
#!pip install sqlalchemy

In [2]:
# importing the library
import sqlalchemy
# checking the version - just for demonstration
sqlalchemy.__version__ 

'1.4.22'

In [3]:
# importing necessary libraries
from sqlalchemy import create_engine

# Engine object refering an in-memory SQLite database
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)

### Database metadata

The central element of both SQLAlchemy Core and ORM is the `SQL Expression Language` which allows for fluent, composable construction of SQL queries. The foundation for these queries are Python objects that represent database concepts like `tables` and `columns`. These objects are known collectively as **database metadata**.

The most common foundational objects for database metadata in SQLAlchemy are known as `MetaData`, `Table`, and `Column`. 

This notebook illustrate how these objects are used in a Core-oriented style, following the reference documentation.

### Core: Setting up MetaData with Table objects
---
In a relational database, the basic structure that we create and query from is known as a `table`. In SQLAlchemy, the *table* is represented by a Python object similarly named `Table`.

To start using the SQLAlchemy Expression Language, we will want to have `Table` objects  that represent all of the database tables we are interested in working with. Each Table may be **declared**, meaning we explicitly spell out in source code what the table looks like, or may be **reflected**, which means we generate the object based on what’s already present in a particular database. The two approaches can also be blended in many ways.

Whether we will declare or reflect our tables, we start out with a collection that will be where we place our tables known as the `MetaData` object. This object is essentially a facade around a Python dictionary that stores a series of `Table` objects keyed to their string name. Constructing this object looks like:


In [4]:
# importing necessary libraries
from sqlalchemy import MetaData

# declaring a metadata object (repository) as a module-level variable to store all tables.
# this is the recommended standard approach for most applications.
metadata = MetaData()

After declaring the `MetaData` object, we can declare `Table` objects. In this tutorial, we will create a table `user`, which represents the users of a website, and the table `address`, representing a list of email addresses associated with rows in the `user` table. Each `Table` object is referred by a variable in the pplication code.


In [5]:
# importing necessary libraries
from sqlalchemy import Table, Column, Integer, String

# creating a table named "user_account" and corresponding columns, referred as "user_table" in the code.
user_table = Table(
    "user_account",  # table name
    metadata,        # metadata (list of columns - name, type and other characteristics)
      Column('id', Integer, primary_key=True),  # SQL datatype Integer
      Column('name', String(30)),  # SQL datatype String, with length of 30
      Column('fullname', String)  # SQL datatype String with no specified length 
    )

The collection of `Column` objects in terms of the parent Table are typically accessed via an associative array located at `Table.c`.


In [6]:
# accessing a specific column object
user_table.c.name

Column('name', String(length=30), table=<user_account>)

In [7]:
# acessing all column objects
user_table.c.keys()

['id', 'name', 'fullname']

###  Core: Declaring primary and foreign key constraints
---

The column `id` in the `user_table` is declared as a *primary_key parameter*. The primary key itself is normally declared implicitly and is represented by the `PrimaryKeyConstraint` construct:



In [8]:
# checking the primary key attribute
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

We can use a `ForeignKeyConstraint` object that corresponds to a database **foreign key constraint** Below we declare a second table address that will have a foreign key constraint referring to the user table:


In [9]:
# importing necessary libraries
from sqlalchemy import ForeignKey

address_table = Table(
    "address",   # table name
    metadata,    # metadata (list of columns - name, type and other characteristics)
      Column('id', Integer, primary_key=True),
      Column('user_id', ForeignKey('user_account.id'), nullable=False),  # definition of foreign key (FK), including the NOT NULL constraint
      Column('email_address', String, nullable=False)
    )

###  Core: Emitting DDL to the Database

So far, we have an object hierarchy to represent two database tables, starting at the root `MetaData` object, then into two `Table` objects, each of which hold onto a collection of `Column` and `Constraint` objects. This object structure will be at the center of most operations we perform with both Core and ORM going forward.

Now, we can emit `CREATE TABLE` statements (or DDL) to our SQLite database so that we can insert and query data from them. We should invoke the `MetaData.create_all()` method on our MetaData, sending it the `Engine` that refers to the target database:


In [10]:
# DDL statement to create all tables
metadata.create_all(engine)

2021-08-16 16:37:38,383 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-16 16:37:38,389 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2021-08-16 16:37:38,391 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-16 16:37:38,394 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2021-08-16 16:37:38,396 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-16 16:37:38,398 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2021-08-16 16:37:38,400 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-16 16:37:38,402 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2021-08-16 16:37:38,403 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-16 16:37:38,407 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2021-08-16 16:37:38,409 INFO sqlalchemy.engine.Engine [no key 0.00197s] ()
2021-08-16 16:37:38,412 INFO sqlalchemy.engine.Engine 
C

`[from documentation]`

The DDL create process includes some `PRAGMA` statements that test for the existence of each table before emitting a `CREATE`. The full series of steps are also included within a `BEGIN/COMMIT` pair to accommodate for transactional DDL.

`CREATE` statements are emitted in the correct order, so `FOREIGN KEY` constraints are guaranteed.

### [OPTIONAL] Combining Core Table declarations with ORM declarative
---

This section is **optional**. Uncomment them (using `Ctrl`  or `Command + /`) if you want to experiment with hybrid approach and table reflection.

We can make use of the `Table` objects we created directly in conjunction with declarative mapped classes from a `declarative_base()` generated base class. This form is called **hybrid table**, and it consists of assigning an existing table definition to the `.__table__` attribute directly, rather than having the declarative process to generate it.

**IMPORTANT:** for the next cell to work, you **should** first run the following sections (in this order):
- Core - Setting up MetaData with Table objects
- Core - Declaring primary and foreign key constraints


In [11]:
# # importing necessary libraries
# from sqlalchemy.orm import declarative_base

# # declarative Base class for mapped classes
# Base = declarative_base()

# # hybrid table using an existing "user_table" mapped to class "User"
# class User(Base):
#   # existing table (previously defined in the code)
#   __table__ = user_table 
#   # relationship mapping ONE user to MANY addressess
#   # this is optional at this point
#   addresses = relationship("Address", back_populates="user")
#   # this method returns a descriptive string of the current class' instance
#   def __repr__(self):
#     return f"User({self.name!r}, {self.fullname!r})"

# # the same applies to an existing "address_table" mapped to class "Address"
# class Address(Base):
#   __table__ = address_table
  
#   user = relationship("User", back_populates="addresses")
  
#   def __repr__(self):
#     return f"Address({self.email_address!r})"

In [12]:
# # checking the Table object associated with a mapped class
# User.__table__

### [OPTIONAL] Table Reflection
---
This section is **optional**. Uncomment them (using `Ctrl`  or `Command + /`) if you want to experiment with hybrid approach and table reflection.

**Table reflection** refers to the process of generating `Table` and related objects by reading the current state of a database. Whereas in the previous sections we’ve been declaring `Table` objects and then emitting DDL to the database, the reflection process does it in reverse.

As an example of reflection, we will create a new object which represents the `some_table` object we created manually in [ex1_sqlalchemy.ipynb](./ex1_sqlalchemy.ipynb). There are again some varieties of how this is performed, however the most basic is i) to construct a Table object, ii) given the name of the table and a MetaData collection to which it will belong, and iii) then instead of indicating individual Column and Constraint objects, pass it the target Engine using the `Table.autoload_with` parameter:

In [13]:
# # [from ex1_sqlalchemy.ipynb] creating the "some_table" object
# from sqlalchemy import text

# with engine.connect() as conn:
#   conn.execute(text("CREATE TABLE some_table (x int, y int)"))
#   conn.execute(
#       text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
#       [{"x": 1, "y": 1}, {"x": 2, "y": 4}]
#       )
#   conn.commit()

In [14]:
# # using reflection to build the "some_table" object from an existing table
# some_table = Table("some_table", metadata, autoload_with=engine)

In [15]:
# # checking the table structure
# some_table

### Core: Inserting Rows 
---
When using Core, a SQL INSERT statement is generated using the `insert()` function - this function generates a new instance of `Insert` which represents an INSERT statement in SQL, that adds new data into a table.

In [16]:
# importing necessary libraries
from sqlalchemy import insert

# creating the INSERT instance (command) to be executed
stmt = insert(user_table).values(name='Spongebob', fullname="Spongebob Squarepants")
# checking how the command is translated into SQL - observe the placeholders (:name, :fullname)
print(stmt)

INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


In [17]:
# we can also check a compiled form of the object which includes a database-specific string SQL representation of the statement.
compiled = stmt.compile()
# placeholders (:name, :fullname) or bound parameters
compiled.params

{'fullname': 'Spongebob Squarepants', 'name': 'Spongebob'}

In [18]:
# executing and commiting the command
with engine.connect() as conn:
  result = conn.execute(stmt)
  conn.commit()

2021-08-16 16:37:38,539 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-16 16:37:38,542 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2021-08-16 16:37:38,545 INFO sqlalchemy.engine.Engine [generated in 0.00584s] ('Spongebob', 'Spongebob Squarepants')
2021-08-16 16:37:38,548 INFO sqlalchemy.engine.Engine COMMIT


The INSERT statement does not return any rows, and if only a single row is inserted, it will usually include the ability to return information about column-level default values that were generated during the INSERT of that row, most commonly an integer primary key value.

In the above case the first row in a SQLite database will normally return 1 for the first integer primary key value, which we can acquire using the `CursorResult.inserted_primary_key` accessor

In [19]:
# checking the returning primary key
result.inserted_primary_key

(1,)

**Executemany** behaviour

In the previous example, we used the `Insert.values()` method to explicitly create the VALUES clause of the SQL INSERT statement. However the usual way that Insert is used is such that the VALUES clause is generated automatically from the parameters passed to the `Connection.execute()` method.

By passing a dictionary or list of dictionaries to the `Connection.execute()` method in conjunction with the Insert construct, the Connection ensures that the column names which are passed will be expressed in the VALUES clause of the Insert construct automatically.


In [20]:
 # inserting data using the "executemany" behaviour
 # the VALUES clause is automatically derived from a dictionary
 with engine.connect() as conn:
   result = conn.execute(
       insert(user_table),
       [
        # dictionary list of values to be inserted in the table
        {"name": "Sandy", "fullname": "Sandy Cheeks"},
        {"name": "Patrick", "fullname": "Patrick Star"}
       ]
    )
   conn.commit()

2021-08-16 16:37:38,579 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-16 16:37:38,583 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2021-08-16 16:37:38,588 INFO sqlalchemy.engine.Engine [generated in 0.00883s] (('Sandy', 'Sandy Cheeks'), ('Patrick', 'Patrick Star'))
2021-08-16 16:37:38,591 INFO sqlalchemy.engine.Engine COMMIT


`[from documentation]`

Below is a more advanced example illustrating how the `Insert.values()` method may be used explicitly while at the same time including for additional VALUES generated from the parameters. A **scalar subquery** is constructed, making use of the `select()` construct, and the parameters used in the subquery are set up using an explicit bound parameter name, established using the `bindparam()` construct.

This allows us to add related rows without fetching the primary key identifiers from the `user_table` operation into the application. 

In [21]:
# importing necessary libraries
from sqlalchemy import select, bindparam

# query to retrieve data from 'user_table" 
scalar_subquery = (
    select(user_table.c.id).
    # bind parameter used in the insert command below
    where(user_table.c.name==bindparam('username')).  
    scalar_subquery()
)

with engine.connect() as conn:
  result = conn.execute(
      insert(address_table).values(user_id=scalar_subquery),
      [
       # each username is passed to the select above to get the ID from each user
       {"username": 'Spongebob', "email_address": "spongebob@sqlalchemy.org"},
       {"username": 'Sandy', "email_address": "sandy@sqlalchemy.org"},
       {"username": 'Sandy', "email_address": "sandy@squirrelpower.org"},
      ]
  )
  conn.commit()

2021-08-16 16:37:38,616 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-16 16:37:38,618 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?), ?)
2021-08-16 16:37:38,621 INFO sqlalchemy.engine.Engine [generated in 0.00511s] (('Spongebob', 'spongebob@sqlalchemy.org'), ('Sandy', 'sandy@sqlalchemy.org'), ('Sandy', 'sandy@squirrelpower.org'))
2021-08-16 16:37:38,626 INFO sqlalchemy.engine.Engine COMMIT


In [22]:
# OPTIONAL - just to check the rows inserted into the "address" table
from sqlalchemy import text

with engine.connect() as conn:
  result = conn.execute(text("SELECT * FROM address"))
  conn.commit()

for row in result:
  print(f"ID:", row.id, "User ID:", row.user_id, "Email:", row.email_address)

2021-08-16 16:37:38,661 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-16 16:37:38,666 INFO sqlalchemy.engine.Engine SELECT * FROM address
2021-08-16 16:37:38,668 INFO sqlalchemy.engine.Engine [generated in 0.00643s] ()
2021-08-16 16:37:38,670 INFO sqlalchemy.engine.Engine COMMIT
ID: 1 User ID: 1 Email: spongebob@sqlalchemy.org
ID: 2 User ID: 2 Email: sandy@sqlalchemy.org
ID: 3 User ID: 2 Email: sandy@squirrelpower.org


**INSERT...FROM.SELECT()**

The `Insert` construct can compose an INSERT that gets rows directly from a SELECT using the `Insert.from_select()` method.

In [23]:
# SELECT command to fetch rows from an existing table...
select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
#...that will be passed into an INSERT command
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
  )
# checking the corresponding SQL commands from the above constructs
print(insert_stmt)

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account


**INSERT...RETURNING**

The `RETURNING` clause for supported backends is used automatically in order to retrieve the last inserted primary key value as well as the values for server defaults. However the `RETURNING` clause may also be specified explicitly using the `Insert.returning()` method; in this case, the Result object that’s returned when the statement is executed has rows which can be fetched:

In [24]:
insert_stmt = insert(address_table).returning(address_table.c.id, address_table.c.email_address)
print(insert_stmt)

INSERT INTO address (id, user_id, email_address) VALUES (:id, :user_id, :email_address) RETURNING address.id, address.email_address


It can also be combined with `Insert.from_select()`, as in the example below that builds upon the example stated in **INSERT...FROM SELECT**.

In [25]:
select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")

insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
  )
print(insert_stmt.returning(address_table.c.id, address_table.c.email_address))

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account RETURNING address.id, address.email_address


### Core: Selecting rows
---
For both Core and ORM, the `select()` function generates a `Select` construct which is used for all SELECT queries. Passed to the method `Connection.execute()` in Core, a SELECT statement is emitted in the current transaction and the result rows available via the returned `Result` object.

The `select()` construct builds up a statement in the same way as that of `insert()`, using a *generative approach* where each method builds more state onto the object. Like the other SQL constructs, it can be stringified in place:

In [26]:
# importing necessary libraries
from sqlalchemy import select

# writing the SELECT statement...
stmt = select(user_table).where(user_table.c.name == 'Spongebob')
#... and checking its translation into SQLite
# Observe the use of placeholders (:name_1)
print(stmt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


As all other statement-level SQL constructs, to *run* the statement we pass it to an execution method. Since a SELECT statement returns rows we can always iterate the result object to get `Row` objects back:

In [27]:
# executing the SELECT statement through the connection
with engine.connect() as conn:
  # execute the statement and iterate over the returning rows (cursor)
  for row in conn.execute(stmt):
    print(row)

2021-08-16 16:37:38,761 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-16 16:37:38,764 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2021-08-16 16:37:38,767 INFO sqlalchemy.engine.Engine [generated in 0.00593s] ('Spongebob',)
(1, 'Spongebob', 'Spongebob Squarepants')
2021-08-16 16:37:38,771 INFO sqlalchemy.engine.Engine ROLLBACK


**Setting the COLUMNS and FROM clause**

The `select()` function accepts positional elements representing any number of Column and/or Table expressions, as well as a wide range of compatible objects, which are resolved into a list of SQL expressions to be SELECTed from that will be returned as columns in the result set. These elements also serve in simpler cases to create the FROM clause, which is inferred from the columns and table-like expressions passed:

In [28]:
# the basic SELECT statement for any Table object
print(select(user_table))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


Individual columns can be accessed from the `Table.c` **accessor** and can be sent directly; the FROM clause will be inferred as the set of all Table and other FromClause objects that are represented by those columns:

In [29]:
# accessing specific colums in the SELECT statement
print(select(user_table.c.name, user_table.c.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


**Selecting from labeled SQL Expressions**

The `ColumnElement.label()` method provides a SQL label of a column or expression, allowing it to have a specific name in a result set. This can be helpful when referring to arbitrary SQL expressions in a result row by name:

In [30]:
# importing necessary libraries
from sqlalchemy import func, cast

# SELECT statement with labeled column
stmt = (
    select(
        ("Username: " + user_table.c.name).label("username"), # column label
    ).order_by(user_table.c.name)
)

# executing the SELECT statement through the connection
with engine.connect() as conn:
  # execute the statement and iterate over the returning rows (cursor)
  for row in conn.execute(stmt):
    # using the label "username" to print the results
    print(f"{row.username}")

2021-08-16 16:37:38,819 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-16 16:37:38,824 INFO sqlalchemy.engine.Engine SELECT ? || user_account.name AS username 
FROM user_account ORDER BY user_account.name
2021-08-16 16:37:38,826 INFO sqlalchemy.engine.Engine [generated in 0.00733s] ('Username: ',)
Username: Patrick
Username: Sandy
Username: Spongebob
2021-08-16 16:37:38,830 INFO sqlalchemy.engine.Engine ROLLBACK


**Selecting with Textual Column Expressions**

Sometimes there is a need to manufacture arbitrary SQL blocks inside of statements, such as constant string expressions, or just some arbitrary SQL that’s quicker to write literally.

The `text()` and `literal_column()` constructs can in fact be embedded into a Select construct directly, such as below where we manufacture a hardcoded string literal 'some label' and embed it within the SELECT statement:

In [31]:
from sqlalchemy import text

stmt = (
    select(
        text("'some phrase'"), user_table.c.name
    ).order_by(user_table.c.name)
)

with engine.connect() as conn:
  print(conn.execute(stmt).all())

2021-08-16 16:37:38,847 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-16 16:37:38,850 INFO sqlalchemy.engine.Engine SELECT 'some phrase', user_account.name 
FROM user_account ORDER BY user_account.name
2021-08-16 16:37:38,853 INFO sqlalchemy.engine.Engine [generated in 0.00623s] ()
[('some phrase', 'Patrick'), ('some phrase', 'Sandy'), ('some phrase', 'Spongebob')]
2021-08-16 16:37:38,856 INFO sqlalchemy.engine.Engine ROLLBACK


In [32]:
from sqlalchemy import literal_column

stmt = (
    select(
        literal_column("'another phrase'").label("p"), user_table.c.name
    ).order_by(user_table.c.name)
)

with engine.connect() as conn:
  for row in conn.execute(stmt):
    print(f"{row.p}, {row.name}")

2021-08-16 16:37:38,876 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-16 16:37:38,879 INFO sqlalchemy.engine.Engine SELECT 'another phrase' AS p, user_account.name 
FROM user_account ORDER BY user_account.name
2021-08-16 16:37:38,881 INFO sqlalchemy.engine.Engine [generated in 0.00492s] ()
another phrase, Patrick
another phrase, Sandy
another phrase, Spongebob
2021-08-16 16:37:38,884 INFO sqlalchemy.engine.Engine ROLLBACK


**The WHERE clause**

Conditional clauses can be composed using standard Python operators, as exemplified below. For boolean expressions, most Python operators such as `==`, `!=`, `<`, `>=` etc. generate new SQL Expression objects, rather than plain boolean `True/False` values:

In [33]:
print(user_table.c.name == 'Squidward')

user_account.name = :name_1


In [34]:
print(address_table.c.user_id > 10)

address.user_id > :user_id_1


We can use expressions like these to generate the WHERE clause by passing the resulting objects to the `Select.where()` method:

In [35]:
print(select(user_table).where(user_table.c.name == 'Squidward'))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


To produce multiple expressions joined by AND, the `Select.where()` method may be invoked any number of times:

In [36]:
print(
    select(address_table.c.email_address).
    where(user_table.c.name == 'Squidward').
    where(address_table.c.user_id == user_table.c.id)
)

SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


“AND” and “OR” conjunctions are both available directly using the `and_()` and `or_()` functions, illustrated below in terms of ORM entities:

In [37]:
from sqlalchemy import and_, or_

print(
    select(address_table.c.email_address).
    where(
        and_(
            or_(user_table.c.name == 'Squidward', user_table.c.name == 'Sandy'),
             address_table.c.user_id == user_table.c.id
        )
    )
)

SELECT address.email_address 
FROM address, user_account 
WHERE (user_account.name = :name_1 OR user_account.name = :name_2) AND address.user_id = user_account.id


For simple *equality* comparisons against a single entity, there’s also a popular method known as `Select.filter_by()` which accepts keyword arguments that match to column keys. It will filter against the leftmost FROM clause or the last entity joined:

In [38]:
print(
    select(user_table).filter_by(name='Spongebob', fullname='Spongebob Squarepants')
)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1 AND user_account.fullname = :fullname_1


**Explicit FROM clauses and JOINs**

The FROM clause is usually inferred based on the expressions that we are setting in the columns clause as well as other elements of the SELECT. If we set a single column from a particular Table in the COLUMNS clause, it puts that Table in the FROM clause as well:

In [39]:
print(select(user_table.c.name))

SELECT user_account.name 
FROM user_account



If we use columns from two tables, then we get a comma-separated FROM clause:

In [40]:
print(select(user_table.c.name, address_table.c.email_address))

SELECT user_account.name, address.email_address 
FROM user_account, address


**Joining tables**

In order to JOIN these two tables together, we typically use one of **two methods** on SELECT.

The first is the `Select.join_from()` method, which allows us to indicate the left and right side of the JOIN explicitly:

In [41]:
print(
    select(user_table.c.name, address_table.c.email_address).
    join_from(user_table, address_table)
)

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


The other is the the `Select.join()` method, which indicates only the right side of the JOIN, the left hand-side is inferred:

In [42]:
print(
    select(user_table.c.name, address_table.c.email_address).
    join(address_table)
)

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


We also have the option to add elements to the FROM clause explicitly, if it is not inferred the way we want from the columns clause. We use the `Select.select_from()` method to achieve this, as below where we establish `user_table` as the first element in the FROM clause and `Select.join()` to establish `address_table` as the second:

In [43]:
print(
    select(address_table.c.email_address).
    select_from(user_table).join(address_table)
)

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


Another example where we might want to use `Select.select_from()` is if our columns clause doesn’t have enough information to provide for a FROM clause. For example, to SELECT from the common SQL expression `count(*)`, we use a SQLAlchemy element known as [sqlalchemy.sql.expression.func](https://docs.sqlalchemy.org/en/14/core/sqlelement.html#sqlalchemy.sql.expression.func) to produce the SQL `count()` function:

In [44]:
# importing necessary libraries
from sqlalchemy import func

print (
    select(func.count('*')).select_from(user_table)
)

SELECT count(:count_2) AS count_1 
FROM user_account


In [45]:
# Example use of sqlalchemy.sql.expression.func
from sqlalchemy import text

with engine.connect() as conn:
  result = conn.execute(select(func.count('*')).select_from(user_table))
  conn.commit()

for row in result:
  print(row)

2021-08-16 16:37:39,091 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-16 16:37:39,095 INFO sqlalchemy.engine.Engine SELECT count(?) AS count_1 
FROM user_account
2021-08-16 16:37:39,098 INFO sqlalchemy.engine.Engine [generated in 0.00654s] ('*',)
2021-08-16 16:37:39,101 INFO sqlalchemy.engine.Engine COMMIT
(3,)


**Setting the ON Clause**

The previous examples of JOIN illustrated that the Select construct can join between two tables and produce the ON clause automatically. This occurs in those examples because the `user_table` and `address_table` Table objects include a single `ForeignKeyConstraint` definition which is used to form this ON clause.

If the left and right targets of the join do not have such a constraint, or there are multiple constraints in place, we need to specify the ON clause directly. Both `Select.join()` and `Select.join_from()` accept an additional argument for the ON clause, which is stated using the same SQL Expression mechanics as we saw about in The WHERE clause:

In [46]:
print(
    select(address_table.c.email_address).
    select_from(user_table).
    join(address_table, user_table.c.id == address_table.c.user_id)
)

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


**OUTER and FULL join**

Both the `Select.join()` and `Select.join_from()` methods accept keyword arguments `Select.join.isouter` and `Select.join.full` which will render LEFT OUTER JOIN and FULL OUTER JOIN, respectively:

Alternativley, there is also a method `Select.outerjoin()` that is equivalent to using `.join(..., isouter=True)`.

In [47]:
print(
    select(user_table).join(address_table, isouter=True)
)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account LEFT OUTER JOIN address ON user_account.id = address.user_id


In [48]:
print(
    select(user_table).join(address_table, full=True)
)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account FULL OUTER JOIN address ON user_account.id = address.user_id


### Core: Aggregate functions
---
Take a look at the [documentation](https://docs.sqlalchemy.org/en/14/tutorial/data_select.html#tutorial-selecting-data) for aggregation functions such as **HAVING, GROUP BY, ORDER BY**.

### Core: Subqueries and CTEs
---
This section will cover a so-called *non-scalar* subquery, which is typically placed in the FROM clause of an enclosing SELECT. We will also cover the *Common Table Expression* (CTE), which is used in a similar way as a subquery, but includes additional features.

SQLAlchemy uses `Select.subquery()` and `Select.cte()` methods to represent a subquery and the CTE, respectively. Either objects can be used as a FROM element inside of a larger `select()` construct.

Example: we can construct a subquery that will select an aggregate count of rows from the `address table`:

In [49]:
subq = select(
    func.count(address_table.c.id).label("count"),
    address_table.c.user_id
  ).group_by(address_table.c.user_id).subquery()

print(subq)

SELECT count(address.id) AS count, address.user_id 
FROM address GROUP BY address.user_id


In [50]:
print(select(subq.c.user_id, subq.c.count))

SELECT anon_1.user_id, anon_1.count 
FROM (SELECT count(address.id) AS count, address.user_id AS user_id 
FROM address GROUP BY address.user_id) AS anon_1


With a selection of rows contained within the `subq` object, we can apply the object to a larger SELECT that will join the data to the `user_account` table:

In [51]:
stmt = select(
    user_table.c.name,
    user_table.c.fullname,
    subq.c.count
).join_from(user_table, subq)

print(stmt)

SELECT user_account.name, user_account.fullname, anon_1.count 
FROM user_account JOIN (SELECT count(address.id) AS count, address.user_id AS user_id 
FROM address GROUP BY address.user_id) AS anon_1 ON user_account.id = anon_1.user_id


In order to join from `user_account` to `address`, we made use of the `Select.join_from()` method. As has been illustrated previously, the ON clause of this join was again inferred based on foreign key constraints. Even though a SQL subquery does not itself have any constraints, SQLAlchemy can act upon constraints represented on the columns by determining that the `subq.c.user_id` column is derived from the `address_table.c.user_id` column, which does express a foreign key relationship back to the `user_table.c.id` column which is then used to generate the ON clause.

**Common Table Expressions (CTEs)**

Usage of the CTE construct in SQLAlchemy is virtually the same as how the Subquery construct is used. By changing the invocation of the `Select.subquery()` method to use `Select.cte()` instead, we can use the resulting object as a FROM element in the same way, but the SQL rendered is the very different common table expression syntax:

In [52]:
subq = select(
    func.count(address_table.c.id).label("count"),
    address_table.c.user_id
).group_by(address_table.c.user_id).cte()

stmt = select(
    user_table.c.name,
    user_table.c.fullname,
    subq.c.count
).join_from(user_table, subq)

print(stmt)

WITH anon_1 AS 
(SELECT count(address.id) AS count, address.user_id AS user_id 
FROM address GROUP BY address.user_id)
 SELECT user_account.name, user_account.fullname, anon_1.count 
FROM user_account JOIN anon_1 ON user_account.id = anon_1.user_id


The CTE construct also features the ability to be used in a *recursive* style, and may in more elaborate cases be composed from the RETURNING clause of an INSERT, UPDATE or DELETE statement. The [docstring for CTE](https://docs.sqlalchemy.org/en/14/core/selectable.html#sqlalchemy.sql.expression.CTE) includes details on these additional patterns.

In both cases, the subquery and CTE were named at the SQL level using an **anonymous** name. In the Python code, we don’t need to provide these names at all. The object identity of the Subquery or CTE instances serves as the syntactical identity of the object when rendered. A name that will be rendered in the SQL can be provided by passing it as the first argument of the `Select.subquery()` or `Select.cte()` methods.

**Scalar and Correlated Subqueries**

A **scalar subquery** is a *subquery that returns exactly zero or one row and exactly one column*. The subquery is then used in the COLUMNS or WHERE clause of an enclosing SELECT statement and is different than a regular subquery in that it is not used in the FROM clause. A **correlated subquery** is a *scalar subquery that refers to a table in the enclosing SELECT statement*.

SQLAlchemy represents the scalar subquery using the `ScalarSelect` construct, which is part of the `ColumnElement` expression hierarchy, in contrast to the regular subquery which is represented by the `Subquery` construct, which is in the FromClause hierarchy.

Scalar subqueries are often, but not necessarily, used with aggregate functions. A scalar subquery is indicated explicitly by making use of the `Select.scalar_subquery()` method as below. It’s default string form when stringified by itself renders as an ordinary SELECT statement that is selecting from two tables:

In [53]:
subq = select(func.count(address_table.c.id)).\
    where(user_table.c.id == address_table.c.user_id).\
    scalar_subquery()

print(subq)

(SELECT count(address.id) AS count_1 
FROM address, user_account 
WHERE user_account.id = address.user_id)


The above `subq` object now falls within the `ColumnElement` SQL expression hierarchy, in that it may be used like any other column expression

In [54]:
print(subq == 5)

(SELECT count(address.id) AS count_1 
FROM address, user_account 
WHERE user_account.id = address.user_id) = :param_1


Although the scalar subquery by itself renders both `user_account` and `address` in its FROM clause when stringified by itself, when embedding it into an enclosing `select()` construct that deals with the user_account table, the user_account table is automatically correlated, meaning it does not render in the FROM clause of the subquery:

In [55]:
stmt = select(user_table.c.name, subq.label("address_count"))
print(stmt)

SELECT user_account.name, (SELECT count(address.id) AS count_1 
FROM address 
WHERE user_account.id = address.user_id) AS address_count 
FROM user_account


Simple correlated subqueries will usually do the right thing that’s desired. However, in the case where the correlation is ambiguous, SQLAlchemy will let us know that more clarity is needed. 

For example, the following query

```
stmt = select(
    user_table.c.name,
    address_table.c.email_address,
    subq.label("address_count")
    ).\
    join_from(user_table, address_table).\
    order_by(user_table.c.id, address_table.c.id)

print(stmt)
```

will generate an error:

```
InvalidRequestError: Select statement '<sqlalchemy.sql.selectable.Select object at 0x7fa3b3efd290>' returned 
no FROM clauses due to auto-correlation; specify correlate(<tables>) to control correlation manually.
```


To specify that the user_table is the one we seek to correlate we specify this using the `ScalarSelect.correlate()` or `ScalarSelect.correlate_except()` methods:

In [56]:
subq = select(func.count(address_table.c.id)).\
  where(user_table.c.id == address_table.c.user_id).\
  scalar_subquery().correlate(user_table)

The statement then can return the data for this column like any other:

In [57]:
with engine.connect() as conn:
  result = conn.execute(
      select(
          user_table.c.name,
          address_table.c.email_address,
          subq.label("address_count")
      ).
      join_from(user_table, address_table).
      order_by(user_table.c.id, address_table.c.id)
  )

print(result.all())

2021-08-16 16:37:39,315 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-16 16:37:39,319 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.email_address, (SELECT count(address.id) AS count_1 
FROM address 
WHERE user_account.id = address.user_id) AS address_count 
FROM user_account JOIN address ON user_account.id = address.user_id ORDER BY user_account.id, address.id
2021-08-16 16:37:39,322 INFO sqlalchemy.engine.Engine [generated in 0.00687s] ()
2021-08-16 16:37:39,325 INFO sqlalchemy.engine.Engine ROLLBACK
[('Spongebob', 'spongebob@sqlalchemy.org', 1), ('Sandy', 'sandy@sqlalchemy.org', 2), ('Sandy', 'sandy@squirrelpower.org', 2)]


### Core: Updating and deleting rows
---
The `update()` function generates a new instance of Update which represents an UPDATE statement in SQL.

Like the `insert()` construct, there is a *traditional* form of `update()`, which emits UPDATE against a single table at a time and does not return any rows. However some backends support an UPDATE statement that may modify multiple tables at once, and the UPDATE statement also supports RETURNING such that columns contained in matched rows may be returned in the result set.

A basic UPDATE looks like:

In [58]:
# importing necessary libraries
from sqlalchemy import update

# defining the UPDATE statement
stmt = (
    update(user_table).where(user_table.c.name == 'Patrick').
    values(fullname='Patrick the Star')
  )
# checking the corresponding SQLite statement
# observe the use of placeholders
print(stmt)

UPDATE user_account SET fullname=:fullname WHERE user_account.name = :name_1


The `Update.values()` method controls the contents of the SET elements of the UPDATE statement. This is the same method shared by the `Insert` construct. Parameters can normally be passed using the column names as keyword arguments.

UPDATE supports all the major SQL forms of UPDATE, including updates against expressions, where we can make use of `Column` expressions:

In [59]:
# update command to set "fullname" as "Username: <name>. Example: Username: Patrick the Star"
stmt = (
    update(user_table).
    values(fullname="Username: " + user_table.c.name)
  )

print(stmt)

UPDATE user_account SET fullname=(:name_1 || user_account.name)


To support UPDATE in an **executemany** context, where many parameter sets will be invoked against the same statement, the `bindparam()` construct may be used to set up bound parameters; these replace the places that literal values would normally go:

In [60]:
from sqlalchemy import bindparam

stmt = (
    update(user_table).
    where(user_table.c.name == bindparam('oldname')).
    values(name=bindparam('newname'))
  )

with engine.begin() as conn:
  conn.execute(
      stmt,
      [
      {'oldname':'jack', 'newname':'ed'},
      {'oldname':'wendy', 'newname':'mary'},
      {'oldname':'jim', 'newname':'jake'},
      ]
  )

2021-08-16 16:37:39,379 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-16 16:37:39,384 INFO sqlalchemy.engine.Engine UPDATE user_account SET name=? WHERE user_account.name = ?
2021-08-16 16:37:39,388 INFO sqlalchemy.engine.Engine [generated in 0.00393s] (('ed', 'jack'), ('mary', 'wendy'), ('jake', 'jim'))
2021-08-16 16:37:39,391 INFO sqlalchemy.engine.Engine COMMIT


**Correlated updates** 

An UPDATE statement can make use of rows in other tables by using a *correlated subquery*. A subquery may be used anywhere a column expression might be placed:

In [61]:
scalar_subq = (
    select(address_table.c.email_address).
    where(address_table.c.user_id == user_table.c.id).
    order_by(address_table.c.id).
    limit(1).
    scalar_subquery() # declares 'scalar_subq' as a correlated subquery 
)

# UPDATE command using a correlated subquery (subq)
update_stmt = update(user_table).values(fullname=scalar_subq)
print(update_stmt)

UPDATE user_account SET fullname=(SELECT address.email_address 
FROM address 
WHERE address.user_id = user_account.id ORDER BY address.id
 LIMIT :param_1)


The `delete()` function generates a new instance of Delete which represents a DELETE statement in SQL, that will delete rows from a table.

The `delete()` statement from an API perspective is very similar to that of the `update()` construct, traditionally returning no rows but allowing for a RETURNING variant on some database backends.

In [62]:
# importing necessary libraries
from sqlalchemy import delete

# defining the DELETE command
stmt = delete(user_table).where(user_table.c.name == 'Patrick')
print(stmt)

DELETE FROM user_account WHERE user_account.name = :name_1


**Getting affected row count from UPDATE and DELETE**

Both `Update` and `Delete` support the ability to return the number of rows matched after the statement proceeds, for statements that are invoked using Core Connection, i.e. `Connection.execute()`. Per the caveats mentioned below, this value is available from the `CursorResult.rowcount` attribute:


In [63]:
with engine.begin() as conn:
  result = conn.execute(
      update(user_table).
      values(fullname="Patrick McStar").
      where(user_table.c.name == 'Patrick')
  )

print(f"Number of affected rows: ", result.rowcount)

2021-08-16 16:37:39,440 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-16 16:37:39,444 INFO sqlalchemy.engine.Engine UPDATE user_account SET fullname=? WHERE user_account.name = ?
2021-08-16 16:37:39,449 INFO sqlalchemy.engine.Engine [generated in 0.00546s] ('Patrick McStar', 'Patrick')
2021-08-16 16:37:39,451 INFO sqlalchemy.engine.Engine COMMIT
Number of affected rows:  1


**Using RETURNING with UPDATE, DELETE**

Like the `Insert` construct, `Update` and `Delete` also support the RETURNING clause which is added by using the `Update.returning()` and `Delete.returning()` methods. When these methods are used on a backend that supports RETURNING, selected columns from all rows that match the WHERE criteria of the statement will be returned in the `Result` object as rows that can be iterated:

In [64]:
update_stmt = (
    update(user_table).where(user_table.c.name == 'Patrick').
    values(fullname='Patrick the Star').
    returning(user_table.c.id, user_table.c.name)
)

print(update_stmt)

UPDATE user_account SET fullname=:fullname WHERE user_account.name = :name_1 RETURNING user_account.id, user_account.name


In [65]:
delete_stmt = (
    delete(user_table).where(user_table.c.name == 'Patrick').
    returning(user_table.c.id, user_table.c.name)
  )

print(delete_stmt)

DELETE FROM user_account WHERE user_account.name = :name_1 RETURNING user_account.id, user_account.name


### Core: Engine disposal
---
See [documentation](https://docs.sqlalchemy.org/en/13/core/connections.html#translation-of-schema-names), section *Engine Disposal* for further details.

`[from documentation]`

The `Engine` refers to a connection pool, which means under normal circumstances, there are open database connections present while the `Engine` object is still resident in memory.

It is not intended to be created and disposed on a per-connection basis; it is instead a registry that maintains both a pool of connections as well as configurational information about the database and DBAPI in use, as well as some degree of internal caching of per-database resources.

However, there are many cases where it is desirable that all connection resources referred to by the Engine be completely closed out. It’s generally not a good idea to rely on Python garbage collection for this to occur for these cases; instead, the Engine can be explicitly disposed using the `Engine.dispose()` method. This disposes of the engine’s underlying connection pool and replaces it with a new one that’s empty. Provided that the Engine is discarded at this point and no longer used, all checked-in connections which it refers to will also be fully closed.


In [66]:
# closing the engine and freeing all allocated resources
engine.dispose()